<a href="https://colab.research.google.com/github/manavmittal05/Large-Language-Models/blob/main/A2/Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Manav Mittal, 2021538
!pip install transformers huggingface_hub
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install bitsandbytes
!pip install accelerate
!pip install datasets

In [ ]:
from huggingface_hub import login
# from google.colab import userdata

# login(userdata.get('HF_TOKEN'))
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")
login(secret_value_0)

In [ ]:
import transformers
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
class Dataset:

  def __init__(self):
    self.dataset = pd.read_parquet("hf://datasets/cais/mmlu/college_mathematics/test-00000-of-00001.parquet")

  def get_zero_shot_prompt(self, idx):
    question = self.dataset.iloc[idx]['question']
    options = self.dataset.iloc[idx]['choices']

    prompt = f"""
    Choose the answer to the given question from below options. No explaination required.

    {question}

    Option 1: {options[0]}
    Option 2: {options[1]}
    Option 3: {options[2]}
    Option 4: {options[3]}

    Answer:"""

    return prompt

  def get_chain_of_thought_prompt(self, idx):
    question = self.dataset.iloc[idx]['question']
    options = self.dataset.iloc[idx]['choices']

    prompt = f"""
    Choose the answer to the given question from below options. Explain the solution step by step.

    {question}

    Option 1: {options[0]}
    Option 2: {options[1]}
    Option 3: {options[2]}
    Option 4: {options[3]}

    Answer:"""

    return prompt


In [ ]:
class llm:

  def __init__(self, model_id):
    self.model_id = model_id
    self.pipeline = transformers.pipeline(
        "text-generation",
        model=self.model_id,
        device_map="auto",
        model_kwargs={
          "torch_dtype": torch.bfloat16,
          "quantization_config": {"load_in_4bit": True},
          "low_cpu_mem_usage": True,
        },

    )

  def generate(self, prompt, max_new_tokens):

    messages = [
        {"role": "user", "content": prompt},
    ]

    outputs = self.pipeline(
        messages,
        max_new_tokens=max_new_tokens,
        # do_sample=True,
        # temperature=0.3,
        # top_p=0.9,
    )
    return outputs[0]["generated_text"][-1]["content"]

In [ ]:
gemma = llm("google/gemma-2b-it")

In [ ]:
pip install -U bitsandbytes

In [ ]:
phi = llm("microsoft/Phi-3.5-mini-instruct")

In [ ]:
llama = llm("meta-llama/Meta-Llama-3.1-8B-Instruct")

In [ ]:
dataset = Dataset()

In [ ]:
from tqdm import tqdm

In [ ]:
# write code to generate answers for the questions in the dataset and save them in a dataframe columns gemma, phi, llama

# df = pd.DataFrame(columns=['gemma', 'phi', 'llama'])

# for i in tqdm(range(len(dataset.dataset))):
#   zero_shot_prompt = dataset.get_zero_shot_prompt(i)

#   gemma_answer = gemma.generate(zero_shot_prompt, 20)
#   phi_answer = phi.generate(zero_shot_prompt, 20)
#   llama_answer = llama.generate(zero_shot_prompt, 20)

#   new_row = {'gemma': gemma_answer, 'phi': phi_answer, 'llama': llama_answer}
#   df.loc[len(df)] = new_row

# # save the zero-shot answers in a csv file
# df.to_csv('zero_shot_answers.csv', index=False)

# write code to generate answers for the questions in the dataset and save them in a dataframe columns gemma, phi, llama

df = pd.DataFrame(columns=['gemma', 'phi', 'llama'])

for i in tqdm(range(len(dataset.dataset))):
  chain_of_thought_prompt = dataset.get_chain_of_thought_prompt(i)

  gemma_answer = gemma.generate(chain_of_thought_prompt, 1024)
  phi_answer = phi.generate(chain_of_thought_prompt, 1024)
  llama_answer = llama.generate(chain_of_thought_prompt, 1024)

  new_row = {'gemma': gemma_answer, 'phi': phi_answer, 'llama': llama_answer}
  df.loc[len(df)] = new_row

# save the chain-of-thought answers in a csv file
df.to_csv('chain_of_thought_answers.csv', index=False)

In [ ]:
print(dataset.get_zero_shot_prompt(0))

In [11]:
print(dataset.get_chain_of_thought_prompt(0))



    Choose the answer to the given question from below options. Explain the solution step by step.



    Let k be the number of real solutions of the equation e^x + x - 2 = 0 in the interval [0, 1], and let n be the number of real solutions that are not in [0, 1]. Which of the following is true? 



    Option 1: k = 0 and n = 1

    Option 2: k = 1 and n = 0

    Option 3: k = n = 1

    Option 4: k > 1

    

    Answer:


In [19]:
print(llama.generate(dataset.get_zero_shot_prompt(0), 100))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Option 1: k = 0 and n = 1


In [20]:
print(gemma.generate(dataset.get_zero_shot_prompt(0), 100))

Option 3: k = n = 1



The equation has one real solution in the interval [0, 1], and there are no real solutions outside of this interval.


In [21]:
print(phi.generate(dataset.get_zero_shot_prompt(0), 100))

 Option 2: k = 1 and n = 0


In [10]:
print(llama.generate(dataset.get_chain_of_thought_prompt(0), 4096))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


To solve this problem, we'll analyze the given equation and understand its behavior.



The given equation is: e^x + x - 2 = 0



We can rewrite this equation as: 



e^x + x = 2  --- (Equation 1)



We'll analyze this equation by considering different cases.



**Case 1: x = 0**



Substituting x = 0 in Equation 1, we get:

e^0 + 0 - 2 = 0

1 + 0 - 2 = 0

-1 ≠ 0



So, x = 0 is not a solution.



**Case 2: x = 1**



Substituting x = 1 in Equation 1, we get:

e^1 + 1 - 2 = 0

e + 1 - 2 = 0

e - 1 ≈ 1.718 - 1 ≈ 0.718

0.718 ≠ 0



So, x = 1 is not a solution.



**Case 3: x > 1**



For x > 1, the function e^x + x is a strictly increasing function. 



When x approaches infinity, e^x + x approaches infinity.



So, there are no solutions for x > 1.



**Case 4: x < 0**



For x < 0, the function e^x + x is a strictly increasing function.



When x approaches -∞, e^x + x approaches -∞.



So, there are no solutions for x < 0.



**Case 5: x = 0.5**



Let's find the value of e^x + x at 

In [11]:
print(gemma.generate(dataset.get_chain_of_thought_prompt(0), 4096))

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


The correct answer is Option 2: k = 1 and n = 0.



**Solution:**



To find the number of real solutions, we can use the fact that the derivative of the equation is equal to 0 at x = 0 and x = 1.



$$\frac{d}{dx}(e^x + x - 2) = e^x + 1 = 0$$



Setting this equal to 0, we get:



$$e^x = -1$$



$$x = - \ln(-1)$$



Substituting this value of x into the original equation, we get:



$$e^{-1} + 1 - 2 = 0$$



$$e^{-1} = 1$$



Therefore, the only real solution is x = 0.



Since the interval [0, 1] contains only one interval of the interval [0, 1], we have:



$$n = 0$$



**Conclusion:**



The answer is Option 2: k = 1 and n = 0.


In [12]:
print(phi.generate(dataset.get_chain_of_thought_prompt(0), 4096))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )

  warnings.warn(

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


 To solve this problem, we need to analyze the behavior of the function f(x) = e^x + x - 2 and determine the number of real solutions within the interval [0, 1], as well as the number of real solutions that are not in this interval.



Step 1: Analyze the function at the endpoints of the interval [0, 1].



f(0) = e^0 + 0 - 2 = 1 - 2 = -1

f(1) = e^1 + 1 - 2 ≈ 2.7183 + 1 - 2 ≈ 0.7183



Since f(0) < 0 and f(1) > 0, and the function is continuous and differentiable on the interval [0, 1], by the Intermediate Value Theorem, there must be at least one real solution in the interval [0, 1].



Step 2: Determine the derivative of the function to analyze its monotonicity.



f'(x) = e^x + 1



Since e^x is always positive, f'(x) = e^x + 1 is also always positive. This means that the function f(x) is strictly increasing on the interval [0, 1].



Step 3: Evaluate the function at the critical point x = 0.



f(0) = e^0 + 0 - 2 = -1



Since f(0) < 0 and f(1) > 0, and the function is strictly in